In [1]:
import pandas as pd
# from tabulate import tabulate
import numpy as np
from pygobnilp.gobnilp import Gobnilp
# import bnlearn as bn
import pgmpy
import itertools
from scipy.stats import chi2_contingency #check for independence between 2 variables
from scipy.stats import power_divergence
from pgmpy.estimators.CITests import log_likelihood
from pgmpy.estimators.CITests import chi_square
# from pgmpy.factors.continuous.discretize import BaseDiscretizer

In [2]:
def csv_format_discrete(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df.drop([0])

#returns the csv_file in a pandas dataframe, formatted properly, discrete dataset only

In [3]:
df = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\discrete.dat")
df
df_test = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\alarm_100.dat")

### chi-squared test

In [4]:
#how do we measure the consistensy
def chi2bool(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.10, boolean=True) #returns chi, p_value, dof
        empty_4.append([j, i , chisquare])
        
    return empty_4

In [5]:
def chi2val(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.05, boolean=False) #returns chi, p_value, dof
        empty_4.append([j, i , chisquare])
        
    return empty_4


In [6]:
#tests whether x is independent of y given a single variable z 
def chi2condbool(df):
    test_list_0 = []
    test_list_1 = []
    test_list_2 = []
    chi2 = []
    v = list(df)
    combine = list(itertools.combinations(v, 3)) #nC3 
    for i in combine:
        test_list_0.append(i[0])
        test_list_1.append(i[1])
        test_list_2.append(i[2])
    for a,b,c in zip(test_list_0, test_list_1, test_list_2):
        chisquare = chi_square(X=a, Y=b, Z=[c], data=df, significance_level=0.05, boolean=True)
        chi2.append([a,b,c,chisquare])
    return sortReturn(chi2)

In [7]:
#tests whether x is independent of y given a single variable z 
def chi2condval(df):
    test_list_0 = []
    test_list_1 = []
    test_list_2 = []
    chi2 = []
    v = list(df)
    combine = list(itertools.combinations(v, 3)) #nC3 
    for i in combine:
        test_list_0.append(i[0])
        test_list_1.append(i[1])
        test_list_2.append(i[2])
    for a,b,c in zip(test_list_0, test_list_1, test_list_2):
        chisquare = chi_square(X=a, Y=b, Z=[c], data=df, significance_level=0.05, boolean=False)
        chi2.append([a,b,c,chisquare])
    return chi2

### log-likelihood tests

In [8]:
def g2val(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        g2 = log_likelihood(X=x, Y=y, Z=[], significance_level=0.05, boolean=True, data=df) 
        empty_4.append([j, i , g2])
        
    return empty_4

### Sorting Functions

In [9]:
def sortReturn(data):
    #just a sorting function
    true_list = []
    false_list = []
    for i in data:
        if True in i:
            true_list.append(i)
        if False in i:
            false_list.append(i)
    return true_list, false_list

In [10]:
#returns a list with a tuple of combinations of false
def false2tuple(data):
    empty = []
    false_list_of_tuples = []
    new_df = pd.DataFrame(data)
    newer_df = new_df[1]
    return newer_df

### CONDITIONAL MAIN CODE

In [11]:
#returns the FALSE values of chi2condbool UNFORMATTED
def conditional_sort_1(df):
    first_letter = []
    second_letter = []
    chi = chi2condbool(df) #returns unsorted list of true and false
    new_chi = chi[1]
    for i in new_chi:
        first_letter.append(i[0])
        second_letter.append(i[1])
    return first_letter, second_letter

In [12]:
#returns FALSE Xs and Ys in a tuple of (X,Y)
def conditional_sort_2(df):
    the_empty = []
    the_list = list(conditional_sort_1(df))
    a = the_list[0]
    b = the_list[1]
    c = len(a)
    for i in range(c):
        the_empty.append((a[i], b[i]))
    return the_empty

In [13]:
#returns combinations of Zs
def conditional_combine_1(df):
    first_empty = []
    second_empty = []
    my_list = conditional_sort_2(df)
    for i in my_list:
        first_empty.append(i[0])
        second_empty.append(i[1])
    list_df = list(df)
    v = list(itertools.combinations(list_df, 2))
    for i,j,k in zip(first_empty, second_empty, v):
        chi_square(X=i, Y=j, Z=k, data=df)
    return chi_square

In [28]:
df_test

,One,Two,Three,Four,Five,Six,Seven,Eight,Nine,Ten,...,TwentyEight,TwentyNine,Thirty,ThirtyOne,ThirtyTwo,ThirtyThree,ThirtyFour,ThirtyFive,ThirtySix,ThirtySeven
1,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0
2,0,2,0,1,2,2,0,1,3,1,...,0,1,1,0,2,0,3,0,0,2
3,0,2,0,1,2,2,0,1,3,1,...,0,0,1,0,1,0,3,0,0,1
4,0,2,0,1,1,1,0,3,3,1,...,1,1,1,0,0,0,3,0,0,2
5,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,1,1,3,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0,2,0,1,2,2,0,1,3,1,...,0,0,1,0,2,0,2,1,0,2
97,0,2,0,1,2,2,0,1,3,1,...,0,0,1,0,0,0,3,0,0,2
98,0,2,0,1,2,2,0,1,3,1,...,2,1,1,0,0,1,3,0,1,0
99,0,2,0,1,2,2,0,1,3,0,...,0,1,1,0,2,0,3,0,0,2


In [22]:
first_empty = []
second_empty = []
my_list = conditional_sort_2(df)
for i in my_list:
    first_empty.append(i[0])
    second_empty.append(i[1])
list_df = list(df)
v = list(itertools.combinations(list_df, 2))
v, first_empty, second_empty
# for i,j,k in zip(first_empty, second_empty, v):
#     chi_square(X=i, Y=j, Z=k, data=df)
# return chi_square

([('A', 'B'),
  ('A', 'C'),
  ('A', 'D'),
  ('A', 'E'),
  ('A', 'F'),
  ('B', 'C'),
  ('B', 'D'),
  ('B', 'E'),
  ('B', 'F'),
  ('C', 'D'),
  ('C', 'E'),
  ('C', 'F'),
  ('D', 'E'),
  ('D', 'F'),
  ('E', 'F')],
 ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'D'],
 ['B', 'B', 'B', 'B', 'C', 'D', 'D', 'E', 'C', 'D', 'D', 'E', 'D', 'D', 'E'])

40.5 µs ± 3.49 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


421 ms ± 43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
chi2condbool(df)[1], conditional_sort_2(df)

In [ ]:
first_empty = []
second_empty = []
my_list = conditional_sort_2(df)
for i in my_list:
    first_empty.append(i[0])
    second_empty.append(i[1])
list_df = list(df)
v = list(itertools.combinations(list_df, 2))
v, first_empty, second_empty

In [ ]:
alpha_test = v[0]
alpha_test
for i,j in zip(first_empty, second_empty):
    if alpha_test[0] != i and alpha_test[0] != j and alpha_test[1] != i and alpha_test[1] != j:
        print(i,j)

In [ ]:
for i,j in zip(v, first_empty):
    print(i[0][0])

In [ ]:
conditional_1P5(df)

In [ ]:
v = list(itertools.combinations(list_df, 2))
v

In [ ]:
m = Gobnilp()

In [ ]:
m.learn(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\discrete.dat")